# Welcome to the Aruna API workshop at the 2024 NFDI4Microbiota annual conference.

<hr/>

**In this workshop you will learn what Aruna is, how the API of Aruna is structured and how you can use it to run your individual data management in Aruna.**

**Be it for your research project or for a service that should use the Aruna services as or in its backend.**

### <ins>Todays Agenda:</ins>

1. **How to install the Aruna Python gRPC API**

2. **How to initiate a client**

3. **How to create an Aruna token**

4. **How to create a Project**

5. **How to create a Collection**

6. **How to create an Object**

7. **How to upload data**

8. **How to enrich an Object with metadata**

    - **With labels**
      
    - **With a relation to another Object that contains the metadata**

10. **How to give other users access to my resources**

11. **How to extend Aruna with hooks**

10. **Aruna v3 Preview**

<hr />

## 1. Install Aruna Python API

Will be likely already installed in this environment, but for the sake of completeness.

In [ ]:
%pip install Aruna-Python-API # Installs the latest Aruna API version. 

## 2. How to initiate a client

### Import service definitions and set global constants

In [ ]:
import grpc
import collections

from aruna.api.storage.services.v2.info_service_pb2_grpc import StorageStatusServiceStub
from aruna.api.storage.services.v2.authorization_service_pb2_grpc import AuthorizationServiceStub
from aruna.api.storage.services.v2.user_service_pb2_grpc import UserServiceStub
from aruna.api.storage.services.v2.project_service_pb2_grpc import ProjectServiceStub
from aruna.api.storage.services.v2.collection_service_pb2_grpc import CollectionServiceStub
from aruna.api.storage.services.v2.dataset_service_pb2_grpc import DatasetServiceStub
from aruna.api.storage.services.v2.object_service_pb2_grpc import ObjectServiceStub
from aruna.api.hooks.services.v2.hooks_service_pb2_grpc import HooksServiceStub

# Define a Aruna Server instance endpoint
#AOS_HOST = 'grpc.aruna.nfdi-dev.gi.denbi.de' # Protocol (e.g. https://) has to be omitted
#AOS_PORT = '443'
AOS_HOST = '0.0.0.0' # Protocol (e.g. https://) has to be omitted
AOS_PORT = '50051'   # Port of the grpc endpoint

API_TOKEN = 'SuperSecretToken'

### Implement automatic authorization

The interceptor automatically adds the specified API token to each request header before sending as: 

`authorization: Bearer <API-Token>`

In [ ]:
class _MyAuthInterceptor(grpc.UnaryUnaryClientInterceptor):
    """
    Implement abstract class grpc.UnaryUnaryClientInterceptor to extend request metadata.
    """
    def intercept_unary_unary(self, continuation, client_call_details, request):
        # Append authorization token to request metadata
        metadata = []
        if client_call_details.metadata is not None:
            metadata = list(client_call_details.metadata)
        metadata.append(('authorization', f'Bearer {API_TOKEN}'))

        # Continue with new client call details
        request_iterator = iter((request,))
        updated_details = _ClientCallDetails(
            client_call_details.method, client_call_details.timeout,
            metadata, client_call_details.credentials
        )

        return continuation(updated_details, next(request_iterator))


class _ClientCallDetails(
    collections.namedtuple('_ClientCallDetails', ('method', 'timeout', 'metadata', 'credentials')),
    grpc.ClientCallDetails):
    """
    Implement grpc.ClientCallDetails to pass modified request details in interceptor.
    """
    pass

### Encapsulate services in class for easier usage

Swagger UI gives an overview of the available API endpoints: [https://api.aruna-engine.org/swagger-ui](https://api.aruna-storage.org/swagger-ui/)

In [ ]:
class ArunaClient(object):
    """
     Class to contain the AOS gRPC client service stubs for easier usage.
    """

    def __init__(self, ):
        #ssl_credentials = grpc.ssl_channel_credentials()
        #self.secure_channel = grpc.secure_channel("{}:{}".format(AOS_HOST, AOS_PORT), ssl_credentials)
        self.insecure_channel = grpc.insecure_channel("{}:{}".format(AOS_HOST, AOS_PORT))
        self.intercept_channel = grpc.intercept_channel(self.insecure_channel, _MyAuthInterceptor())

        self.info_client = StorageStatusServiceStub(self.intercept_channel)
        self.auth_client = AuthorizationServiceStub(self.intercept_channel)
        self.user_client = UserServiceStub(self.intercept_channel)
        self.project_client = ProjectServiceStub(self.intercept_channel)
        self.collection_client = CollectionServiceStub(self.intercept_channel)
        self.dataset_client = DatasetServiceStub(self.intercept_channel)
        self.object_client = ObjectServiceStub(self.intercept_channel)
        self.hook_client = HooksServiceStub(self.intercept_channel)

        # self.other_client = ...

client = ArunaClient()

### Get Status Information

This is a good request to test the client configuration as it does not need any authentication.

In [ ]:
# Imports from generated gRPC modules
from aruna.api.storage.services.v2.info_service_pb2 import GetPubkeysRequest

# Create tonic/ArunaAPI request to fetch user info of current user
get_pubkeys_request = GetPubkeysRequest()

# Send the request to the AOS instance gRPC gateway
get_pubkeys_response = client.info_client.GetPubkeys(request=get_pubkeys_request)

# Do something with the response
print(f'{get_pubkeys_response}')

## 3. How to create an Aruna token

1. Login to supported OIDC identity provider: https://dev.aruna-engine.org

2. Use OIDC token to register user in Aruna

3. Create Aruna token through the web interface or the API

In [ ]:
API_TOKEN = '<OIDC-Token>'

# Imports from generated gRPC modules
from aruna.api.storage.services.v2.user_service_pb2 import RegisterUserRequest

# Create tonic/ArunaAPI request to fetch user info of current user
register_user_request = RegisterUserRequest()

# Send the request to the AOS instance gRPC gateway
register_user_response = client.info_client.GetPubkeys(request=register_user_request)

# Do something with the response
print(f'{register_user_response}')

In [ ]:
import datetime

# Imports from generated gRPC modules
from aruna.api.storage.services.v2.user_service_pb2 import CreateAPITokenRequest
from google.protobuf.internal.well_known_types import Timestamp

# Create tonic/ArunaAPI request to fetch user info of current user
create_token_request = CreateAPITokenRequest(
    name = "My Aruna token",
    expires_at = datetime.datetime(2030, 1, 1, 0, 0, 0)
)

# Send the request to the AOS instance gRPC gateway
create_token_response = client.user_client.CreateAPIToken(request=create_token_request)

# Do something with the response
print(f'{create_token_response}')

## Get User Information

In [ ]:
# Imports from generated gRPC modules
from aruna.api.storage.services.v2.user_service_pb2 import GetUserRequest

# Create tonic/ArunaAPI request to fetch user info of current user
get_user_request = GetUserRequest()

# Send the request to the AOS instance gRPC gateway
get_user_response = client.user_client.GetUser(request=get_user_request)

# Do something with the response
print(f'{get_user_response}')

## 3. How to create a Project

### Create a Project

In [ ]:
# Imports from generated gRPC modules
from aruna.api.storage.services.v2.project_service_pb2 import CreateProjectRequest
from aruna.api.storage.models.v2.models_pb2 import DataClass

# Create tonic/ArunaAPI request to fetch information of a project
create_project_request = CreateProjectRequest(
    name = "my-workshop-project",
    title = "My Workshop Project",
    description = "A workshop project created with the gRPC Python API client.",
    authors = [
    ],
    key_values = [], 
    relations = [], 
    data_class = DataClass.DATA_CLASS_PUBLIC,
    preferred_endpoint = "",
    metadata_license_tag = "AllRightsReserved",
    default_data_license_tag = "AllRightsReserved"
)

# Send the request to the AOS instance gRPC gateway
create_project_response = client.project_client.CreateProject(request=create_project_request)

# Do something with the response
PROJECT_ID = create_project_response.project.id
print(f'{create_project_request}')

**Current Project State:**

![Project with admin user](./imgs/Demo_empty_project.jpg)

## Get Project Information

In [ ]:
# Imports from generated gRPC modules
from aruna.api.storage.services.v2.project_service_pb2 import GetProjectRequest

# Create tonic/ArunaAPI request to fetch information of a project
get_project_request = GetProjectRequest(
    project_id = PROJECT_ID
)

# Send the request to the AOS instance gRPC gateway
get_project_response = client.project_client.GetProject(request=get_project_request)

# Do something with the response
print(f'{get_project_response}')

## Create Collection for Objects

In [ ]:
# Imports from generated gRPC modules
from aruna.api.storage.services.v2.collection_service_pb2 import CreateCollectionRequest
from aruna.api.storage.models.v2.models_pb2 import DataClass, KeyValue

# Create tonic/ArunaAPI request to fetch user info of current user
create_collection_request = CreateCollectionRequest(
        project_id=PROJECT_ID, # Parent of the collection
        name = "my-workshop-collection",
        title = "My Workshop Collection",
        description = "A workshop collection created with the gRPC Python API client.",
        key_values = [],
        relations=[],
        data_class=DataClass.DATA_CLASS_PUBLIC,
        metadata_license_tag="AllRightsReserved",
        default_data_license_tag="AllRightsReserved",
        authors=[]
    )

# Send the request to the AOS instance gRPC gateway
create_collection_response = client.collection_client.CreateCollection(request=create_collection_request)

# Do something with the response
COLLECTION_ID = create_collection_response.collection.id
print(f'{create_collection_response}')

## Get Collection Information

In [ ]:
# Imports from generated gRPC modules
from aruna.api.storage.services.v2.collection_service_pb2 import GetCollectionRequest

# Create tonic/ArunaAPI request to fetch user info of current user
get_collection_request = GetCollectionRequest(
    collection_id = COLLECTION_ID
)

# Send the request to the AOS instance gRPC gateway
get_collection_response = client.collection_client.GetCollection(request=get_collection_request)

# Do something with the response
print(f'{get_collection_response}')


**Current Project State:**

![Project with admin user](./imgs/Demo_simple_collection.jpg)

## Create Object and fill it with data

Normally, the creation of an Object consists of three steps:

1. Initialize the Object
2. Upload the data
3. Finish the Object (will be performed automatically for a single part upload)

These three steps are required independently to avoid inconsistencies in the storage.

### Initialize Object

In [ ]:
# Imports from generated gRPC modules
from aruna.api.storage.services.v2.object_service_pb2 import CreateObjectRequest
from aruna.api.storage.models.v2.models_pb2 import Hash, Hashalgorithm

# Create tonic/ArunaAPI request to initialize an staging object
init_data_object_request = CreateObjectRequest(
    name = "GCF_000005845.2_ASM584v2_genomic.fna",
    title = "Genome assembly ASM584v2",
    description = "Escherichia coli str. K-12 substr. MG1655, complete genome.",
    key_values = [],
    relations = [],
    data_class = DataClass.DATA_CLASS_PUBLIC,
    #project_id="<project-id>",
    collection_id = COLLECTION_ID,
    #dataset_id="<dataset-id>",
    hashes = [],
    metadata_license_tag="AllRightsReserved",
    data_license_tag="AllRightsReserved",
    authors=[]
)

# Send the request to the AOS instance gRPC gateway
init_data_object_response = client.object_client.CreateObject(request=init_data_object_request)

# Do something with the response
OBJECT_ID = init_data_object_response.object.id
print(f'{init_data_object_response}')

### Upload single part data

In [ ]:
import gzip
import requests

# Imports from generated gRPC modules
from aruna.api.storage.services.v2.object_service_pb2 import GetUploadURLRequest

# Create tonic/ArunaAPI request to request an upload url for single part upload
get_upload_url_request = GetUploadURLRequest(
    object_id=OBJECT_ID,
    multipart=False,
    part_number=1
)

# Send the request to the AOS instance gRPC gateway
get_upload_url_response = client.object_client.GetUploadURL(request=get_upload_url_request)

# Do something with the response
print(f'{get_upload_url_response}')
upload_url = get_upload_url_response.url

# Load remote file content in memory
remote_file_url = "https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/005/845/GCF_000005845.2_ASM584v2/GCF_000005845.2_ASM584v2_genomic.fna.gz"
remote_file_gzipped = requests.get(remote_file_url).content

with gzip.open(remote_file_gzipped, 'rb') as gzipped_file:
    # Upload file content to the generated upload URL
    headers = {'Content-type': 'application/octet-stream'}
    upload_response = requests.put(upload_url, data=gzipped_file.read(), headers=headers)

    # Do something with the response
    print(f'{upload_response}')

### Finish Object (Only for multipart uploads)

In [ ]:
import hashlib

# Imports from generated gRPC modules
from aruna.api.storage.services.v2.object_service_pb2 import FinishObjectRequest

# Calculate SHA256 Hashsum
sha256_file_hash = hashlib.sha256(remote_file_raw).hexdigest()

# Create tonic/ArunaAPI request to finish a single part upload staging object
finish_request = FinishObjectStagingRequest(
    object_id     = init_first_object_response.object_id,
    upload_id     = init_first_object_response.upload_id,
    collection_id = init_first_object_response.collection_id,
    hash = Hash(
        alg  = Hashalgorithm.Value("HASHALGORITHM_MD5"),
        hash = md5_file_hash
    ),
    auto_update = True
)

# Send the request to the AOS instance gRPC gateway
finish_response = client.object_client.FinishObjectStaging(request=finish_request)

# Do something with the response
print(f'{finish_response}')

### All three steps together for metadata object

In [ ]:
https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/005/845/GCF_000005845.2_ASM584v2/GCF_000005845.2_ASM584v2_assembly_report.txt

# Create tonic/ArunaAPI request to initialize an staging object
init_meta_object_request = InitializeNewObjectRequest(
    object=StageObject(
        filename      = "GCF_000005845.2_ASM584v2_assembly_report.txt",
        description   = "Accompanying metadata for Genome assembly ASM584v2",
        content_len   = 1207,
        labels        = [KeyValue(key="data_object", value=init_data_object_response.object_id)],
        dataclass     = DataClass.Value("DATA_CLASS_PRIVATE")
    ),
    collection_id = create_collection_response.collection_id
)

# Send the request to the AOS instance gRPC gateway
init_meta_object_response = client.object_client.InitializeNewObject(request=init_meta_object_request)

# Do something with the response
print(f'{init_meta_object_response}')

# Create tonic/ArunaAPI request to request an upload url for single part upload
get_upload_url_request = GetUploadURLRequest(
    object_id     = init_meta_object_response.object_id,
    upload_id     = init_meta_object_response.upload_id,
    collection_id = init_meta_object_response.collection_id,
    part_number   = 1 # Increases in case of multipart upload
)

# Send the request to the AOS instance gRPC gateway
get_upload_url_response = client.object_client.GetUploadURL(request=get_upload_url_request)

# Do something with the response
upload_url = get_upload_url_response.url.url

# Load remote file content in memory
remote_file_url = "https://gitlab-pe.gwdg.de/gfbio/ammod-examples-schemas/-/raw/master/examples/weather/raw%20data/WSC11_07212669_20220131T175851_metadata.json"
remote_file_raw = requests.get(remote_file_url).content

# Upload file content to the generated upload URL
headers = {'Content-type': 'application/octet-stream'}
upload_response = requests.put(upload_url, data=remote_file_raw, headers=headers)

# Do something with the response
upload_response.raise_for_status

# Calculate MD5 Hashsum of file
md5_file_hash = hashlib.md5(remote_file_raw).hexdigest()

# Create tonic/ArunaAPI request to finish a multipart upload staging object
finish_request = FinishObjectStagingRequest(
    object_id     = init_meta_object_response.object_id,
    upload_id     = init_meta_object_response.upload_id,
    collection_id = init_meta_object_response.collection_id,
    hash = Hash(
        alg  = Hashalgorithm.Value("HASHALGORITHM_MD5"),
        hash = md5_file_hash
    ),
    auto_update = True
)

# Send the request to the AOS instance gRPC gateway
finish_response = client.object_client.FinishObjectStaging(request=finish_request)

# Do something with the response
print(f'{finish_response}')


**Current Project State:**

![Project with admin user](./imgs/Demo_simple_objects.jpg)

## Create ObjectGroup to bundle data and metadata

In [ ]:
# Imports from generated gRPC modules
from aruna.api.storage.services.v1.objectgroup_service_pb2 import CreateObjectGroupRequest

# Create tonic/ArunaAPI request to create a new object group
create_objectgroup_request = CreateObjectGroupRequest(
    name            = "WSC11_07212669_20220131T175851 ObjectGroup",
    description     = "WSC11_07212669_20220131T175851 data/metadata bundle",
    collection_id   = create_collection_response.collection_id,
    object_ids      = [init_first_object_response.object_id],
    meta_object_ids = [init_meta_object_response.object_id],
    labels          = [KeyValue(
        key="sensor_id",
        value="34254915479139543"
    )]
)

# Send the request to the AOS instance gRPC gateway
create_objectgroup_response = client.object_group_client.CreateObjectGroup(request=create_objectgroup_request)

# Do something with the response
print(f'{create_objectgroup_response}')

## Get ObjectGroup Information

In [ ]:
# Imports from generated gRPC modules
from aruna.api.storage.services.v1.objectgroup_service_pb2 import GetObjectGroupByIdRequest

# Create tonic/ArunaAPI request to fetch information about a specific object group
get_objectgroup_request = GetObjectGroupByIdRequest(
    group_id      = create_objectgroup_response.object_group.id,
    collection_id = create_collection_response.collection_id
)

# Send the request to the AOS instance gRPC gateway
get_objectgroup_response = client.object_group_client.GetObjectGroupById(request=get_objectgroup_request)

# Do something with the response
print(f'{get_objectgroup_response}')

In [ ]:
# Imports from generated gRPC modules
from aruna.api.storage.services.v1.objectgroup_service_pb2 import GetObjectGroupsRequest
from aruna.api.storage.models.v1.query_pb2 import LabelOrIDQuery, LabelFilter

# Create tonic/ArunaAPI request to fetch information about a specific object group
get_objectgroups_request = GetObjectGroupsRequest(
        collection_id = create_collection_response.collection_id,
        label_id_filter = LabelOrIDQuery(
            labels = LabelFilter(
                labels = [KeyValue(key="sensor_id", value="34254915479139543")]
            )
        )
    )

# Send the request to the AOS instance gRPC gateway
get_objectgroups_response = client.object_group_client.GetObjectGroups(request=get_objectgroups_request)

# Do something with the response
print(f'{get_objectgroups_response}')


**Current Project State:**

![Project with admin user](https://www.computational.bio.uni-giessen.de/nextcloud/index.php/s/M6Zf6993LnWGmgq/download/Demo_simple_objectgroup.jpg)

## Get Objects of ObjectGroup

In [ ]:
# Imports from generated gRPC modules
from aruna.api.storage.services.v1.objectgroup_service_pb2 import GetObjectGroupObjectsRequest, ObjectGroupObject
from aruna.api.storage.services.v1.object_service_pb2 import GetDownloadURLRequest

# Create tonic/ArunaAPI request to fetch information of the first 20 objects of an object group including meta objects
get_objects_request = GetObjectGroupObjectsRequest(
    group_id      = create_objectgroup_response.object_group.id,
    collection_id = create_collection_response.collection_id
)

# Send the request to the AOS instance gRPC gateway
get_objects_response = client.object_group_client.GetObjectGroupObjects(request=get_objects_request)

# Do something with the response
print(f"{get_objects_response}")

In [ ]:
data_object_ids     = [dobject.object.id for dobject in get_objects_response.object_group_objects if dobject.is_metadata == False]
metadata_object_ids = [mobject.object.id for mobject in get_objects_response.object_group_objects if mobject.is_metadata == True]

print(f'{data_object_ids}')
print(f'{metadata_object_ids}')

## Download data Objects

In [ ]:
import os
import json

from urllib.parse import urlparse, parse_qs

# Imports from generated gRPC modules
from aruna.api.storage.services.v1.object_service_pb2 import GetDownloadURLRequest

# Download all data objects
for data_object_id in data_object_ids:
    # Create tonic/ArunaAPI request to fetch an Objects download url
    download_url_request = GetDownloadURLRequest(
        collection_id = create_collection_response.collection_id,
        object_id     = data_object_id
    )

    # Send the request to the AOS instance gRPC gateway
    download_url_response = client.object_client.GetDownloadURL(request=download_url_request)

    # Extract download url from response
    download_url = download_url_response.url.url

    # Download data
    with requests.get(download_url) as response:
        response.raise_for_status # Raise error if response contains error status
        
        json_body = json.loads(response.content)
        print(json.dumps(json_body, indent=2))

## Download metadata Objects

In [ ]:
# Download all metadata objects
for metadata_object_id in metadata_object_ids:
    # Create tonic/ArunaAPI request to fetch an Objects download url
    download_url_request = GetDownloadURLRequest(
        collection_id = create_collection_response.collection_id,
        object_id     = metadata_object_id
    )

    # Send the request to the AOS instance gRPC gateway
    download_url_response = client.object_client.GetDownloadURL(request=download_url_request)

    # Extract download url from response
    download_url = download_url_response.url.url

    # Download data
    with requests.get(download_url) as response:
        response.raise_for_status # Raise error if response contains error status

        json_body = json.loads(response.content)
        print(json.dumps(json_body, indent=2))

# Extended Feature Demonstration
<hr/>

- Object reusal
- Object referencing

<hr/>

## Upload another weather data point and reuse e.g. station metadata

In [ ]:
# Create tonic/ArunaAPI request to initialize an staging object
init_second_object_request = InitializeNewObjectRequest(
    object=StageObject(
        filename      = "WSC11_07212669_20220131T181543.json",
        description   = "Another weather data point.",
        content_len   = 1100,
        dataclass     = DataClass.Value("DATA_CLASS_PRIVATE")
    ),
    collection_id = create_collection_response.collection_id
)

# Send the request to the AOS instance gRPC gateway
init_second_object_response = client.object_client.InitializeNewObject(request=init_second_object_request)

# Do something with the response
print(f'{init_second_object_response}')

# Create tonic/ArunaAPI request to request an upload url for single part upload
get_upload_url_request = GetUploadURLRequest(
    object_id     = init_second_object_response.object_id,
    upload_id     = init_second_object_response.upload_id,
    collection_id = init_second_object_response.collection_id,
    part_number   = 1 # Increases in case of multipart upload
)

# Send the request to the AOS instance gRPC gateway
response = client.object_client.GetUploadURL(request=get_upload_url_request)

# Do something with the response
upload_url = response.url.url

# Load remote file content in memory
remote_file_url = "https://gitlab-pe.gwdg.de/gfbio/ammod-examples-schemas/-/raw/master/examples/weather/raw%20data/WSC11_07212669_20220131T175851.json"
remote_file_raw = requests.get(remote_file_url).content
updated_remote_file_raw = remote_file_raw.replace(b'971.9000244140625', b'980.9000244140625').replace(b'2022-01-31T17:58:51+0100', b'2022-01-31T18:15:43+0100')

# Upload file content to the generated upload URL
headers = {'Content-type': 'application/octet-stream'}
upload_response = requests.put(upload_url, data=updated_remote_file_raw, headers=headers)

# Do something with the response
upload_response.raise_for_status

# Calculate MD5 Hashsum of file
md5_file_hash = hashlib.md5(updated_remote_file_raw).hexdigest()

# Create tonic/ArunaAPI request to finish a multipart upload staging object
finish_request = FinishObjectStagingRequest(
    object_id     = init_second_object_response.object_id,
    upload_id     = init_second_object_response.upload_id,
    collection_id = init_second_object_response.collection_id,
    hash = Hash(
        alg  = Hashalgorithm.Value("HASHALGORITHM_MD5"),
        hash = md5_file_hash
    ),
    auto_update = True
)

# Send the request to the AOS instance gRPC gateway
finish_response = client.object_client.FinishObjectStaging(request=finish_request)

# Do something with the response
print(f'{finish_response}')

## Bundle new data point with existing metadata in ObjectGroup

In [ ]:
# Create tonic/ArunaAPI request to create a new object group
create_objectgroup_request = CreateObjectGroupRequest(
    name            = "WSC11_07212669_20220131T181543 ObjectGroup",
    description     = "WSC11_07212669_20220131T181543 data/metadata bundle",
    collection_id   = create_collection_response.collection_id,
    object_ids      = [init_second_object_response.object_id],
    meta_object_ids = [init_meta_object_response.object_id],
    labels          = [KeyValue(
        key="sensor_id",
        value="34254915479139543"
    )]
)

# Send the request to the AOS instance gRPC gateway
create_objectgroup_response = client.object_group_client.CreateObjectGroup(request=create_objectgroup_request)

# Do something with the response
print(f'{create_objectgroup_response}')

## Display Object IDs of both ObjectGroups

In [ ]:
# Create tonic/ArunaAPI request to fetch information about a specific object group
get_objectgroups_request = GetObjectGroupsRequest(
        collection_id = create_collection_response.collection_id,
        label_id_filter = LabelOrIDQuery(
            labels = LabelFilter(
                labels = [KeyValue(key="sensor_id", value="34254915479139543")]
            )
        )
    )

# Send the request to the AOS instance gRPC gateway
get_objectgroups_response = client.object_group_client.GetObjectGroups(request=get_objectgroups_request)

# Do something with the response
for object_group in get_objectgroups_response.object_groups.object_group_overviews:
    # Create tonic/ArunaAPI request to fetch information of the first 20 objects of an object group including meta objects
    get_objects_request = GetObjectGroupObjectsRequest(
        group_id      = object_group.id,
        collection_id = create_collection_response.collection_id
    )

    # Send the request to the AOS instance gRPC gateway
    response = client.object_group_client.GetObjectGroupObjects(request=get_objects_request)

    # Print all object ids contained in object group
    print(f"Object IDs: {[og_object.object.id for og_object in response.object_group_objects]}")

**Current Project State**

![Current Project State](./imgs/Generic/Demo_simple_second_data.jpg)

## Reference Objects in another Collection

In [ ]:
# Create tonic/ArunaAPI request to create another collection
create_target_collection_request = CreateNewCollectionRequest(
    project_id  = PROJECT_ID,
    name        = "Raw Weather Data",
    description = "Collection with references to all raw weather data points.",
    dataclass   = DataClass.Value("DATA_CLASS_PRIVATE")
)

# Send the request to the AOS instance gRPC gateway
create_target_collection_response = client.collection_client.CreateNewCollection(request=create_target_collection_request)

# Do something with the response
print(f'{create_target_collection_response}')

In [ ]:
# Imports from generated gRPC modules
from aruna.api.storage.services.v1.object_service_pb2 import CreateObjectReferenceRequest

# Create tonic/ArunaAPI request to reference first object in another collection
create_reference_request = CreateObjectReferenceRequest(
    object_id            = init_first_object_response.object_id,
    collection_id        = create_collection_response.collection_id,
    target_collection_id = create_target_collection_response.collection_id,
    writeable            = False, # Read Only
    auto_update          = True # Always updates to latest version of source object
)

# Send the request to the AOS instance gRPC gateway
create_reference_response = client.object_client.CreateObjectReference(request=create_reference_request)

# Do something with the response
print(f'{create_reference_response}')

In [ ]:
# Create tonic/ArunaAPI request to reference second object in another collection
create_reference_request = CreateObjectReferenceRequest(
    object_id            = init_second_object_response.object_id,
    collection_id        = create_collection_response.collection_id,
    target_collection_id = create_target_collection_response.collection_id,
    writeable            = False, # Read Only
    auto_update          = True # Always updates to latest version of source object
)

# Send the request to the AOS instance gRPC gateway
create_reference_response = client.object_client.CreateObjectReference(request=create_reference_request)

# Do something with the response
print(f'{create_reference_response}')

**Current Project State**

![Current Project State](https://www.computational.bio.uni-giessen.de/nextcloud/index.php/s/QqKwr8Qy2sSadye/download/Demo_simple_reference.jpg)